In [207]:
import numpy as np

# 任务数量
task_number = 90
# cloudlet数量
cloudlet_number = 20

# 任务大小 服从 均值=4600M条指令 的 均匀分布
task_size = np.random.uniform(500,8700,task_number).round()
# 移动设备处理能力(MIPS)
speed_moblieDevice = 600
# cloudlet处理能力(MIPS)
speed_cloudlet = 3846
# clooud处理能力(MIPS)
speed_cloud = 31790

# 任务所需传输数据量 服从 均值=750KB 的 均匀分布 假设下行的时间为上行的 0.6 倍
task_upload = np.random.uniform(200,1300,task_number).round()
# 边缘传输速率：带宽设为100mbps 理论传输率最大 12.8MB/s  实际传输吞吐量 4-11MB/S
trans_rate_clt = np.random.uniform(4,11,task_number).round(2)
# 云传输速率：带宽设为1Gbps 理论传输率最大 128MB/s  实际传输吞吐量 40-110MB/S
trans_rate_c = np.random.uniform(40,110,task_number).round(2)

power_idle = 5
power_communication = 20
power_excute = 60
# 总的时间 和 能耗
Tm = Tclt = Tc = 0
Em = Eclt = Ec = 0

# 失败任务的个数
failureCount = 0

In [208]:
for i in range(task_number):

    # if i <= task_number * 0.05:
    #     # print('在本地执行')
    #     temp = task_size[i] / speed_moblieDevice
    #     if temp > 5:
    #         failureCount += 1
    #     Tm = Tm + temp
    #     Em = Em + power_excute * temp
    # elif task_number * 0.05 < i <= task_number * 0.35:
    if i <= task_number * 0.35:
        # 在cloudlet1上 通信时间比较久
        # print('cloudlet0')
        cloudlet0_upload = (task_upload[i] / 1024) / trans_rate_clt[i]
        cloudlet0_propagation = np.random.randint(5,50) / 3e8
        cloudlet0_queue = np.random.uniform(0,1.5)
        cloudlet0_excute = task_size[i] / speed_cloudlet
        cloudlet0_oneTaskTime = 8*cloudlet0_upload + cloudlet0_propagation + cloudlet0_queue + cloudlet0_excute 
        if cloudlet0_oneTaskTime > 5:
            failureCount += 1
        Tclt += cloudlet0_oneTaskTime
        Eclt = Eclt +  power_communication * cloudlet0_upload * 8 + power_idle * (cloudlet0_propagation + cloudlet0_queue + cloudlet0_excute)
    elif  task_number * 0.35 < i <= task_number * 0.65:
        # 传播到cloudlet2的时间比较久 且执行慢
        # print('cloudlet1')
        cloudlet1_upload = (task_upload[i] / 1024) / trans_rate_clt[i]
        cloudlet1_propagation = np.random.randint(5,50) / 3e8
        cloudlet1_queue = np.random.uniform(0,1.5)
        cloudlet1_excute = task_size[i] / (speed_cloudlet * 0.8)
        cloudlet1_oneTaskTime = cloudlet1_upload + 40*cloudlet1_propagation + cloudlet1_queue + cloudlet1_excute 
        if cloudlet1_oneTaskTime > 5:
            failureCount += 1
        Tclt += cloudlet1_oneTaskTime
        Eclt = Eclt +  power_communication * cloudlet1_upload * 1.6 + power_idle * (40*cloudlet1_propagation + cloudlet1_queue + cloudlet1_excute)
    elif task_number * 0.65 < i < task_number * 0.95:
        # print('cloudlet2')
        # 在cloudlet2的排队时间比较久
        cloudlet2_upload = (task_upload[i] / 1024) / trans_rate_clt[i]
        cloudlet2_propagation = np.random.randint(5,50) / 3e8
        cloudlet2_queue = np.random.uniform(1,2)
        cloudlet2_excute = task_size[i] / speed_cloudlet
        cloudlet2_oneTaskTime = cloudlet2_upload + cloudlet2_propagation + cloudlet2_queue + cloudlet2_excute 
        if cloudlet2_oneTaskTime > 5:
            failureCount += 1
        Tclt += cloudlet2_oneTaskTime
        Eclt = Eclt +  power_communication * cloudlet2_upload * 1.6 + power_idle * (cloudlet2_propagation + cloudlet2_queue + cloudlet2_excute)
    else:
        # print('cloud')
        Tc_upload = (task_upload[i] / 1024) / trans_rate_c[i]
        Tc_propagation = np.random.randint(10000,500000) / 3e8
        Tc_excute = task_size[i] / speed_cloud
        if Tc_upload * 1.6 + Tc_propagation  + Tc_excute > 5:
            failureCount += 1
        Tc = Tc +  Tc_upload * 1.6 + Tc_propagation  + Tc_excute
        Ec = Ec + power_communication * Tc_upload * 1.6 + power_idle * (Tc_propagation + Tc_excute)
        
print("执行这些任务的时间约为：",round(Tm + Tclt + Tc,2))
print("执行这些任务的能耗约为：", round(Em + Eclt + Ec,2))
print("任务失败个数为：",failureCount)

执行这些任务的时间约为： 223.77
执行这些任务的能耗约为： 1651.12
任务失败个数为： 1
